In [ ]:
# Initialize the simulation parameters
def main():
    gamma_L = 2
    gamma_R = 3

    eps = 1.0

    mu_L= 1
    mu_R= 2

    # Temperature of Reserviors
    T_L = 1
    T_R= 1

    time = 1.5
    dt = 0.6
    nt = int(time/dt)

    beta_L=1/T_L
    beta_R=1/T_R
    F_L = 1/(np.exp(beta_L*(eps-mu_L))+1)
    F_R = 1/(np.exp(beta_R*(eps-mu_R))+1)

    # Generate Hamiltonians
    exact_diag_ham = build_exact_diag_hamiltonian(eps)
    ham_real, ham_imag = hamiltonian_generation(eps, gamma_L, gamma_R, F_R,F_L)



    # Generate initial states
    vqte_init_state, exact_diag_init_state, ansatz, init_param_values = build_initial_states(ham_real, ansatz)
    print("VQTE Initial State : " , vqte_init_state)
    print("Exact Initial : ", exact_diag_init_state)

    # Perform exact diagonalization
    exact_diag_results, time_points = perform_exact_diag(gamma_L, F_L, gamma_R, F_R, dt, nt, exact_diag_init_state, exact_diag_ham)

    # Perform vqte simulation
    vqte_results = perform_vqte(ham_real, ham_imag, vqte_init_state, dt, nt, ansatz, init_param_values)


    output_results(
    vqte_results, 
    exact_diag_results, 
    time_points, 
    mu_L, T_L, gamma_L, 
    mu_R, T_R, gamma_R)




main()



TypeError: build_initial_states() missing 1 required positional argument: 'ansatz'